In [234]:
import pandas as pd

In [235]:
def read_data(file_path):
    df = pd.read_csv(file_path)
    columns_to_keep = ['resources.description', 'gross_amount', 'gocardless_fees', 'app_fees', 'net_amount', 'payouts.arrival_date', 'payments.metadata.Member', 'payments.metadata.References']
    df = df[columns_to_keep]
    return df


In [236]:
def split_years(df):
    df['year'] = df['resources.description'].str.extract(r'(\d{4})')
    df['resources.description'] = df['resources.description'].str.replace(r'\d{4}', '', regex=True)
    df['year'] = df['year'].fillna(0).astype(int)

    df['year_2'] = df['resources.description'].str.extract(r'(\d{2})')
    df['resources.description'] = df['resources.description'].str.replace(r'\d{2}', '', regex=True)
    df['year_2'] = pd.to_datetime(df['year_2'], format='%y').dt.strftime('%Y')
    df['year_2'] = df['year_2'].fillna(0).astype(int)

    df['year'] = df['year'] + df['year_2']

    df = df.drop(['year_2'], axis=1)

    return df


In [237]:
def clean_subscriptions_data(subset, schedule):
    subset['schedule'] = schedule
    subset['resources.description'] = subset['resources.description'].replace({'\\(|\\)': '', '  ': ' '}, regex=True)
    subset['resources.description'] = subset['resources.description'].str.split(':', expand=False)
    subset['section_month'] = subset['resources.description'].str[1]
    subset['section_month'] = subset['section_month'].str.split(' ', expand=False)
    subset['section_month'] = subset['section_month'].apply(lambda x: [item for item in x if item])
    subset['section'] = subset['section_month'].str[0]
    subset['payment_name'] = subset['section_month'].str[1]
    subset = subset.drop(['section_month'], axis=1)

    return subset


In [238]:
def clean_activities_data(subset, schedule):
    subset['schedule'] = schedule
    subset['resources.description'] = subset['resources.description'].replace({'\\(|\\)': '', '  ': ' '}, regex=True)
    subset['resources.description'] = subset['resources.description'].str.split(':', expand=False)
    subset['payment_name'] = subset['resources.description'].str[1]
    subset['section'] = subset['payment_name'].str.split().str[0]
    subset['payment_name'] = subset['payment_name'].str.strip()

    return subset

In [239]:
def strip_metadata(df):
    df[['payment_code', 'schedule_code', 'section_code']] = df['payments.metadata.References'].str.split('-', expand=True)
    df[['payment_code', 'schedule_code', 'section_code']] = df[['payment_code', 'schedule_code', 'section_code']].apply(lambda x: x.str.strip())
    return df
    # split out names

In [240]:
def clean_member_names(df):
    df['payments.metadata.Member'] = df['payments.metadata.Member'].str.split('(', expand=False)
    df['member'] = df['payments.metadata.Member'].str[0]
    return df

In [241]:
def clean_data(df):
    # df = read_data(file_path)

    df = split_years(df)
   
    payment_schedules = ['Subscriptions', 'Activities']

    dfs = []

    # for loops through teh schedules and treats each differently
    for schedule in payment_schedules:
        subset = df[df['resources.description'].str.contains(schedule)].copy()
        if schedule == 'Subscriptions':
            subset = clean_subscriptions_data(subset=subset, schedule=schedule)
        elif schedule == 'Activities':
            subset = clean_activities_data(subset=subset, schedule=schedule)
        dfs.append(subset)

    df = pd.concat(dfs, ignore_index=True)
    
    df = strip_metadata(df)
    df = clean_member_names(df)
    df['total_fees'] = df['gocardless_fees'] + df['app_fees']
        
    # General tidy up
    df = df.drop(['resources.description','payments.metadata.References', 'gocardless_fees', 'app_fees', 'payments.metadata.Member'], axis=1)
    column_order = ['section', 'schedule', 'year', 'payment_name', 'gross_amount', 'total_fees', 'net_amount', 'member',
       'payouts.arrival_date', 'section_code', 'schedule_code', 'payment_code']
    
    df = df[column_order]

    return df

In [242]:
def create_metadata(index, subset, date):
    
    id = index+1
    payments_num = subset.shape[0]+1
    date = pd.to_datetime(date).strftime('%d-%b-%Y')
    amount = round(subset['gross_amount'].sum(),2)

    output_str = f"""
    ID: {id}
    Number of Payments: {payments_num}
    Date of Payout: {date}
    Payout Amount: £{amount}
    """
    return output_str

In [243]:
def group_data(df):

    dfs = []

    for idx, date in enumerate(df['payouts.arrival_date'].unique()):
        subsets = []
        subset_date = df[df['payouts.arrival_date'] == date]
        meta_data = create_metadata(index=idx, subset=subset_date, date=date)
        print(meta_data)
        for schedule in ['Subscriptions', 'Activities']:
            subset_schedule = subset_date[subset_date['schedule'] == schedule]
            if schedule == 'Subscriptions':
                subset_subs = subset_schedule.groupby(['section', 'schedule'])[['gross_amount', 'total_fees', 'net_amount']].sum().reset_index()
                subsets.append(subset_subs)

            elif schedule == 'Activities':
                subset_subs = subset_schedule.groupby(['section', 'schedule', 'payment_name'])[['gross_amount', 'total_fees', 'net_amount']].sum().reset_index()
                subsets.append(subset_subs)
        subset_date = pd.concat(subsets, ignore_index=False)
        subset_date = subset_date.fillna('')
        row_order = {
                    'Squirrels' : 1,
                    'Beavers' : 2,
                    'Cubs' : 3,
                    'Scouts' : 4
                    }
        subset_date['no.'] = subset_date['section'].map(row_order)

        column_order = ['no.', 'schedule', 'section', 'payment_name', 'gross_amount', 'total_fees', 'net_amount']
        subset_date = subset_date[column_order]

        subset_date = subset_date.sort_values(by=['schedule', 'no.', 'payment_name'], ascending=[False, True, True]).set_index(keys=['schedule', 'section', 'payment_name'])
        subset_date = subset_date.drop(columns='no.', axis=1)
        dfs.append(subset_date)
        display(subset_date)
    return dfs

In [244]:
def break_down_payouts(file_path):
    df = read_data(file_path)
    df = clean_data(df)
    dfs = group_data(df)
    return dfs

file_path='inputs/payout_transactions_reconciliation-export-EX00036AZABPPE.csv'
dfs = break_down_payouts(file_path=file_path)    


    ID: 1
    Number of Payments: 142
    Date of Payout: 12-Feb-2024
    Payout Amount: £2527.0
    


gross_amount  \
schedule      section   payment_name                                        
Subscriptions Squirrels                                             180.0   
              Beavers                                               288.0   
              Cubs                                                  348.0   
              Scouts                                                540.0   
Activities    Beavers   Beavers Summer Camp Adventure                30.0   
              Cubs      Cubs Christmas Camp 2nd installment          25.0   
                        Cubs Summer Camp Adventure                   90.0   
              Scouts    Scouts  Log Cabin                           696.0   
                        Scouts Summer Adventure Camp                330.0   

                                                             total_fees  \
schedule      section   payment_name                                      
Subscriptions Squirrels                                            7.80   
              Beavers                                             12.48   
              Cubs                                                15.08   
              Scouts                                              23.40   
Activities    Beavers   Beavers Summer Camp Adventure              1.05   
              Cubs      Cubs Christmas Camp 2nd installment        0.90   
                        Cubs Summer Camp Adventure                 3.15   
              Scouts    Scouts  Log Cabin                         22.08   
                        Scouts Summer Adventure Camp              11.55   

                                                             net_amount  
schedule      section   payment_name                                     
Subscriptions Squirrels                                          172.20  
              Beavers                                            275.52  
              Cubs                                               332.92  
              Scouts                                             516.60  
Activities    Beavers   Beavers Summer Camp Adventure             28.95  
              Cubs      Cubs Christmas Camp 2nd installment       24.10  
                        Cubs Summer Camp Adventure                86.85  
              Scouts    Scouts  Log Cabin                        673.92  
                        Scouts Summer Adventure Camp             318.45